In [1]:
# Celda 1: Importaciones
!pip install -q kokoro>=0.9.2 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
import shared_variable
import time
from kokoro import KPipeline
from IPython.display import display, Audio, HTML
import soundfile as sf
import pygame
import threading

El sistema no puede encontrar la ruta especificada.


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\kage-\Downloads\Trabajo - clase\KC - IA estudios\KC_TFG_AudioBook\env\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
def inicializar_sistema():
    """Inicializa el sistema como lo haría el notebook"""
    print("🔗 Inicializando sistema...")

    try:
        pygame.mixer.init()
        print("pygame_disponible")
    except:
        print("pygame_disponible = False")

    # Inicializar la variable
    valor_inicial = shared_variable.initialize_variable('Valor Inicial desde Script')
    print(f"📊 Variable inicializada con: '{valor_inicial}'")
    
    return valor_inicial

def mostrar_variable():
    """Muestra el valor actual de la variable"""
    valor = shared_variable.get_variable()
    print(f"📊 Valor actual de la variable: '{valor}'")
    return valor

def cambiar_variable(nuevo_valor):
    """Cambia la variable"""
    resultado = shared_variable.set_variable(nuevo_valor)
    print(f"✅ Variable cambiada a: '{resultado}'")
    return resultado

def ejecutar_logica_personalizada():
    """Aquí puedes poner la lógica específica de tu notebook"""
    print("🚀 Ejecutando lógica personalizada...")
    
    valor_actual = shared_variable.get_variable()
    
    # Procesar el valor (ejemplo)
    if isinstance(valor_actual["prompt"], str):
        pipeline = KPipeline(lang_code=valor_actual["language"])
        text = valor_actual["prompt"]
        generator = pipeline(text, voice='af_heart',split_pattern=r'\n+')
        for i, (gs, ps, audio) in enumerate(generator):
            print(i, gs, ps)
            display(Audio(data=audio, rate=24000, autoplay=True))
            sf.write(f'{i}.wav', audio, 24000)
            pygame.mixer.music.load(f'{i}.wav')
            pygame.mixer.music.play()
            print("🔊 Reproduciendo audio...")
            while pygame.mixer.music.get_busy():
                pygame.time.wait(100)
            print("✅ Audio finalizado")
    
    print("✅ Lógica personalizada completada")

def monitorear_cambios(duracion=10):
    """Monitorea cambios en la variable"""
    print(f"👀 Monitoreando cambios durante {duracion} segundos...")
    valor_anterior = shared_variable.get_variable()
    tiempo_inicio = time.time()
    
    while time.time() - tiempo_inicio < duracion:
        valor_actual = shared_variable.get_variable()
        if valor_actual != valor_anterior:
            print(f"🔄 ¡Variable cambió! '{valor_anterior}' -> '{valor_actual}'")
            valor_anterior = valor_actual
        time.sleep(1)
    
    print("⏰ Monitoreo completado")

def main():
    """Función principal que se ejecuta cuando se llama al script"""
    print("=" * 50)
    print("🐍 EJECUTANDO SCRIPT DEL NOTEBOOK")
    print("=" * 50)
    
    # Inicializar
    inicializar_sistema()
    
    # Mostrar valor actual
    mostrar_variable()
    
    # Ejecutar lógica personalizada
    ejecutar_logica_personalizada()
    
    # Mostrar valor final
    print("\n📈 Estado final:")
    mostrar_variable()
    
    print("\n✅ Script completado exitosamente!")
    print("=" * 50)

if __name__ == "__main__":
    main()


# Celda 5: Widget interactivo (opcional)
try:
    from ipywidgets import interact, widgets
    from IPython.display import display
    
    def widget_cambiar_variable(nuevo_valor=""):
        """Widget interactivo para cambiar la variable"""
        if nuevo_valor.strip():
            shared_variable.set_variable(nuevo_valor)
            print(f"✅ Variable actualizada desde widget: '{nuevo_valor}'")
        else:
            print(f"📊 Valor actual: '{shared_variable.get_variable()}'")
    
    # Crear widget
    print("\n🎛️ Widget interactivo disponible:")
    interact(widget_cambiar_variable, nuevo_valor=widgets.Text(
        placeholder="Introduce el nuevo valor...",
        description="Variable:"
    ))
    
except ImportError:
    print("⚠️ ipywidgets no disponible. Instala con: pip install ipywidgets")


🐍 EJECUTANDO SCRIPT DEL NOTEBOOK
🔗 Inicializando sistema...
pygame_disponible
📊 Variable inicializada con: '{'prompt': 'Que día es hoy?\n\n¿Qué día de la semana es hoy?\n```python\nimport datetime\n\ntoday = datetime.date.today()\nday_of_week = today.strftime("%A")\n\nprint(day_of_week)\n```\n', 'language': 'e'}'
📊 Valor actual de la variable: '{'prompt': 'Que día es hoy?\n\n¿Qué día de la semana es hoy?\n```python\nimport datetime\n\ntoday = datetime.date.today()\nday_of_week = today.strftime("%A")\n\nprint(day_of_week)\n```\n', 'language': 'e'}'
🚀 Ejecutando lógica personalizada...


C:\Users\kage-\Downloads\Trabajo - clase\KC - IA estudios\KC_TFG_AudioBook\env\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
C:\Users\kage-\Downloads\Trabajo - clase\KC - IA estudios\KC_TFG_AudioBook\env\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


0 Que día es hoy? ke ðˈia ˈes ˈoɪ?


🔊 Reproduciendo audio...


✅ Audio finalizado


...

✅ Audio finalizado


6 print(day_of_week) pɾˈint(dˈI ˈof weˈek)


🔊 Reproduciendo audio...


✅ Audio finalizado
✅ Lógica personalizada completada

📈 Estado final:
📊 Valor actual de la variable: '{'prompt': 'Que día es hoy?\n\n¿Qué día de la semana es hoy?\n```python\nimport datetime\n\ntoday = datetime.date.today()\nday_of_week = today.strftime("%A")\n\nprint(day_of_week)\n```\n', 'language': 'e'}'

✅ Script completado exitosamente!

🎛️ Widget interactivo disponible:


interactive(children=(Text(value='', description='Variable:', placeholder='Introduce el nuevo valor...'), Outp…

In [3]:
# Distorsión de voz usando RVC y pydub
import os
from pydub import AudioSegment
from subprocess import run
from pydub.effects import speedup

# --- CONFIGURACIÓN ---
input_audio = "0.wav"                    # Voz original
output_audio = "voz_convertida_rvc.wav"      # Voz convertida sin efectos
final_output = "voz_final_distorsionada.wav" # Voz con efectos

model_path = "weights/mi_modelo.pth"
index_path = "logs/mi_modelo/added_IVF.index"
config_path = "configs/config.json"          # Usa el que corresponda (40k o 48k)

# --- 1. CONVERSIÓN DE VOZ (usa RVC CLI) ---
# command = [
#     "python", "infer.py", 
#     "-m", model_path,
#     "-i", input_audio,
#     "-o", output_audio,
#     "--index", index_path,
#     "--f0", "crepe",
#     "--pitch", "0",        # Puedes poner -5 o +5 para cambiar el tono
#     "--method", "harvest", # harvest o crepe (elige el que mejor te suene)
#     "--config", config_path
# ]
# run(command)

# --- 2. DISTORSIÓN POSTPROCESADA ---
if not os.path.exists(input_audio):
    print(f"❌ El archivo de audio '{input_audio}' no existe. Asegúrate de que la conversión se realizó correctamente.")
    exit(1)
else:
    audio = AudioSegment.from_file(input_audio)

    distorted = (
        audio
        .low_pass_filter(500)
        .high_pass_filter(1800)
        .speedup(playback_speed=1.3)
        # .reverse()
        .apply_gain(6) 
    )
    alien_layer = distorted - 15
    alien_layer = alien_layer.speedup(playback_speed=0.7)

    final = distorted.overlay(alien_layer)
    final.export(final_output, format="wav")
    print(f"✅ Voz distorsionada guardada en: {final_output}")

✅ Voz distorsionada guardada en: voz_final_distorsionada.wav


C:\Users\kage-\Downloads\Trabajo - clase\KC - IA estudios\KC_TFG_AudioBook\env\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
